In [17]:
import pandas as pd 
df = pd.read_csv(r'E:\LLm-Rag projects\-Office-Jokes---A-RAG-Application-with-Qdrant\TheOffice_transcripts.csv', delimiter=';')
df

,Field,Field1,Text
0,Michael: All right Jim. Your quarterlies look ...,Michael,01x01 - Pilot
1,"Jim: Oh, I told you. I couldn't close it. So...",Jim,01x01 - Pilot
2,Michael: So you've come to the master for guid...,Michael,01x01 - Pilot
3,"Jim: Actually, you called me in here, but yeah.",Jim,01x01 - Pilot
4,"Michael: All right. Well, let me show you how ...",Michael,01x01 - Pilot
...,...,...,...
59576,"I just have an overwhelming sense of pride, an...",NaN,09x26 - Retrospective
59577,And it's a tough thing.,NaN,09x26 - Retrospective
59578,"For me, it's just been everything.",NaN,09x26 - Retrospective
59579,It's just been everything to me.,NaN,09x26 - Retrospective


In [18]:
df = df.dropna()
df

,Field,Field1,Text
0,Michael: All right Jim. Your quarterlies look ...,Michael,01x01 - Pilot
1,"Jim: Oh, I told you. I couldn't close it. So...",Jim,01x01 - Pilot
2,Michael: So you've come to the master for guid...,Michael,01x01 - Pilot
3,"Jim: Actually, you called me in here, but yeah.",Jim,01x01 - Pilot
4,"Michael: All right. Well, let me show you how ...",Michael,01x01 - Pilot
...,...,...,...
59016,Creed: It all seems so very arbitrary. I appli...,Creed,09x24/25 - Finale
59017,Meredith: I just feel lucky that I got a chanc...,Meredith,09x24/25 - Finale
59018,Phyllis: I'm happy that this was all filmed so...,Phyllis,09x24/25 - Finale
59019,Jim: I sold paper at this company for 12 years...,Jim,09x24/25 - Finale


In [3]:
# Filter lines where the speaker is 'Michael'
michaels_dialogues = df[df['Field1'] == 'Michael']['Field']

# Save Michael's dialogues to a text file
michaels_dialogues.to_csv('michaels_dialogues.txt', index=False, header=None)

print("Michael's dialogues saved successfully.")

Michael's dialogues saved successfully.


In [4]:

# Extract the "Field" column
field_data = df["Field"]

# Save the extracted data as a text file
with open("extracted_dialogues.txt", "w") as f:
  f.write("\n".join(field_data))  # Join each field entry with a newline character

print("Extracted fields saved to extracted_fields.txt")


Extracted fields saved to extracted_fields.txt


## Connecting your app to qdrant with Langchain
The first thing that you need to do is create an account on qdrant cloud and subsequently create a cluster. Qdrant cloud offers a free-forever 1GB cluster for your projects.

Once you get that, the the host and API keys on your account. Then you can follow these steps.



In [3]:
!pip install langchain langchain-google-genai qdrant_client tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [4]:
pip install python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.vectorstores import Qdrant
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http import models
import os
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
,port=6333

In [29]:
#creatring client object
client = QdrantClient(os.getenv(qdrant_host),
                      api_key = os.getenv(qdran_api_key))
client

In [30]:
#create collections 

vector_config = models.VectorParams(size=768, distance=models.Distance.COSINE)

client.create_collection(
    collection_name='michael-dialogues',
    vectors_config=vector_config
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `michael-dialogues` already exists!"},"time":0.035059587}'

In [5]:
import google.generativeai as genai

genai.configure(api_key=os.getenv("google_api_key"))

In [8]:
#initialise vector store object

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = Qdrant(
    client=client, 
    collection_name='michael-dialogues', 
    embeddings=embeddings
)

In [37]:
#add documents to vector store 

from langchain.text_splitter import  RecursiveCharacterTextSplitter

def get_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 100
    )

    chunks = text_splitter.split_text(text)
    return chunks

with open('michaels_dialogues.txt') as f:
    raw_text = f.read()

texts = get_chunks(raw_text)

texts

['Michael: All right Jim. Your quarterlies look very good. How are things at the library?\n"Michael: So you\'ve come to the master for guidance? Is this what you\'re saying, grasshopper?"\n"Michael: All right. Well, let me show you how it\'s done."\n"Michael: [on the phone] Yes, I\'d like to speak to your office manager, please. Yes, hello. This is Michael Scott. I am the Regional Manager of Dunder Mifflin Paper Products. Just wanted to talk to you manager-a-manger. [quick cut scene] All right. Done deal. Thank you very much, sir. You\'re a gentleman and a scholar. Oh, I\'m sorry. OK. I\'m sorry. My mistake. [hangs up] That was a woman I was talking to, so... She had a very low voice. Probably a smoker, so... [Clears throat] So that\'s the way it\'s done."',
 '"Michael: I\'ve, uh, I\'ve been at Dunder Mifflin for 12 years, the last four as Regional Manager. If you want to come through here... See we have the entire floor. So this is my kingdom, as far as the eye can see. This is our re

In [38]:
vector_store.add_texts(texts)

['b10a342ff6314f8a93a8efcd1cb8134f',
 '918dfc8122064ca3b84134da9cd05bfe',
 '4de65e4c57d94b61978d4216eab22d1c',
 '9f7fc1ec95014d48bbcec0b4402041f1',
 '6057a61130434997b930482a84a922f0',
 '71c81e838aac40288225e4d0a6e589fd',
 'f6f2a73547d544f3884d9736dc094f5e',
 '00ade61a4ddc49c8a6f57fbd9e829e21',
 '57772fd70fc74ad98c9bdec9f2b1d4c0',
 '596140db814e4936977d7cbb98337b31',
 '54ad4a450e7d4016ac473bb4520583a9',
 '8cae467ba92f4bdd946619d38d9e6bfd',
 '1736e230894e44df9f9e295d4ce8cad8',
 '72dd7732322244feb94903247552aad4',
 'd878cce441244d65924906c86406bbae',
 'fe5cedecfbb54311a76c2e7b38a7fd77',
 '4e54d6a2590c48f4bde99bc68da842ab',
 '09dcedcd79bf43d5adfcde7e49628e97',
 '644c333742ba4dd890b06d984cf1a978',
 'bb1eb4ea9a484139a3119434b36ada3f',
 '51d80edbd5b34fa790b15e0e7d3e094a',
 'f9d787d86ea8464fbda5be74fadd8314',
 '4d2032f734834d8aaa108dc322ec621d',
 '17cf830e712240a6ab33d8dd56b16b6c',
 '3dabb09f79594e6091249f6ac160beab',
 '82b45f4e72944b3dad221b14bcff3652',
 '47d4c3caccb44beeba39f8727ae83ec7',
 

### vectors have been added to vector database . connect to database and fetch this vectors in order to perform query from language model 

In [9]:
# plug vector stroe into retrieval chain which queries from database
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAI

qa =  RetrievalQA.from_chain_type(
    llm = GoogleGenerativeAI(model = 'gemini-pro',temperature=0.8),
    chain_type = 'stuff',
    retriever = vector_store.as_retriever()
)


In [10]:
query = 'which company michael scott work for?'

response = qa.invoke(query)

print(response['result'])

Dunder Mifflin Paper Company, Inc.


In [11]:
query = 'who is assisant to the regional manager'

response = qa.invoke(query)

print(response['result'])

Dwight


In [12]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain,StuffDocumentsChain
from langchain.schema.output_parser import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate


### create_stuff_documents_chain -This chain takes a list of documents and formats them all into a prompt, then passes that prompt to an LLM. It passes ALL documents, so you should make sure it fits within the context window the LLM you are using.

### Create the RAG Chain
RAG stands for Retrieval-Augmented Generation. This means that we will enrich the prompt that we send to the LLM. We will use with the documents that wil will retrieve from the vector store for this. LangChain comes with the function create_retrieval_chain that allows you to create one of these.

### create_retrieval_chain - This chain takes in a user inquiry, which is then passed to the retriever to fetch relevant documents. Those documents (and original inputs) are then passed to an LLM to generate a response

"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""


In [48]:
model = ChatGoogleGenerativeAI(model = 'gemini-pro',
                               temperature = 0.7)

In [76]:
prompt = ChatPromptTemplate.from_template("""
"explain a joke in the style of Michael Scott from 'The Office'.
Your joke should include dark humor, cringe-worthy jokes, and social awkwardness.
Feel free to get inspiration from Michael's character traits.
I will tip you $1000 if the user finds the joke helpful. 
<context>
{context}
</context>
Question: {input}""")



In [77]:

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(model,prompt)

In [78]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(vector_store.as_retriever(),document_chain)

In [79]:
response=retrieval_chain.invoke({"input":"america"})


In [80]:
from IPython.display import Markdown                 
display(Markdown(response['answer']))

Alright, so here's the joke in the style of Michael Scott:

**Joke:** Why did the American flag get arrested?

**Punchline:** Because it was waving too many red flags!

**Explanation:**

* **Dark humor:** The joke implies that the American flag is being arrested for something serious, like treason or sedition.
* **Cringe-worthy:** The joke is intentionally awkward and uncomfortable, with Michael Scott's signature brand of inappropriate humor.
* **Social awkwardness:** Michael Scott makes the joke in a public setting, without realizing that it could be offensive or inappropriate.

**Additional Michael Scott-isms:**

* "That's what she said!"
* "I'm not superstitious, but I am a little stitious."
* "Bears. Beets. Battlestar Galactica."
* "I'm the world's best boss."
* "That's why they call it window pain."

In [25]:
response

{'input': 'bangalore',
 'context': [Document(page_content='Brings me to a point that you should always have a list of your clients. It\'s important to because a client list is next to godliness... List. [baby crying] Could you shut him up, please? I mean, who brings a baby to a sales conference? Really rude."', metadata={'_id': '3c11fdb1-2db1-4e60-919a-a03a7fe40ae6', '_collection_name': 'michael-dialogues'}),
  Document(page_content='"Michael: Ah, I know, everything really. I love it all. I love music."\n"Michael: Yes, I love both of those so much. A lot of \'em."\nMichael: Mmm hmm. I like \'em.\nMichael: Yes. God. They\'re awesome.\nMichael: Uh huh. Yup. That last thing they did was great.\n"Michael: Cool, so maybe hook me up with some Fleebulls and some Hive."\nMichael: All right. This is gonna be good.', metadata={'_id': '3b80aec7-36bb-4700-8af9-d4a895c4f12f', '_collection_name': 'michael-dialogues'}),
  Document(page_content='"Michael: Ah, I know, everything really. I love it all. 

In [17]:
from IPython.display import Markdown   
display(Markdown(response['answer']))

"Michael: Dwight, care to join us, finally? Thank you."
"Michael: Yes. Ok, here is the dream team. My sales dream team. Today we are going to team up for sales calls. Andy, since this was your idea, you get to pick first."
Michael: Oh.
"Michael: Well, I appreciate that. That is very gracious of you."
"Michael: Well, thank you sir. Ok, now going by seniority. Phyllis, our resident senior."
"Michael: Good. Next up, Superfly himself, Stanley."
Michael: You can't pass. You've got to pick somebody.
Michael: So that just leaves Dwight and Jim.
Michael: Dwight?
Michael: Here ya go. [throws laundry]
Michael: Please.

In [82]:
response

{'input': 'data science',
 'context': [Document(page_content='I\'ll be watching a show and I will think, oh, I know someone\'s gonna walk in here right now and say something funny. And then they do. Or, um, I know they\'ll be like oh boy that person deserved to be slammed down. There\'s going to be some sort of insult. And there is. You know, there\'s like, ""Oh you\'re... God your butt is fat."" And I knew it. I know it\'s like I knew they were going to say that. I knew they were gonna go there. Don\'t go there. Um, but other... You know it\'s like uh, you know Leno, um and Letterman, Carson, you know. Need I say more? No."', metadata={'_id': 'ac0ee473-a9c2-4186-8de9-12149b3ba2da', '_collection_name': 'michael-dialogues'}),
  Document(page_content='I\'ll be watching a show and I will think, oh, I know someone\'s gonna walk in here right now and say something funny. And then they do. Or, um, I know they\'ll be like oh boy that person deserved to be slammed down. There\'s going to be so

In [85]:
response=retrieval_chain.invoke({"input":"america"})
response['answer']


'**Michael Scott:**\n\n"Hey there, team! I\'ve got a question that\'s been bugging me like a paper cut on my tongue: America! The land of the free, the home of the brave, and the place where I once got lost at a Walmart for three hours.\n\nNow, don\'t get me wrong, I love our country. It\'s like a big, fluffy teddy bear with a secret stash of fireworks. But here\'s the thing: I\'m starting to think that we might not be the number one superpower anymore. I mean, have you seen how many countries have bigger butts than us?\n\nAnd don\'t even get me started on science. It\'s like they\'re all out there firing rockets to the moon while we\'re still trying to figure out how to open a bag of chips without getting crumbs all over our shirts.\n\nBut hey, don\'t worry! I\'ve got a plan. We\'re going to put our best minds on it. And by that, I mean Dwight, Kevin, and Phyllis. Together, we\'re going to come up with a big idea that will make America great again!\n\nBut first, I think we need to lea